In [20]:
%pylab inline
import pandas as pd
import os 

Populating the interactive namespace from numpy and matplotlib


Construcción base de datos:

In [21]:
#Inicializo las columnas con los nombres de las localidades que ya tienen datos
localidades=os.listdir("Votaciones_localidades")
Nombreslocalidades=[]
for i in localidades:
    Nombreslocalidades.append(i[:-7])
df1 = pd.DataFrame(columns=[Nombreslocalidades])
df1= df1.rename_axis(None)


In [22]:
#Algoritmo que ingresa los datos a la tabla, la tabla solo contiene las localidades reportadas.
Index=('P_Liberal','P_Conservador','P_Opcion_Ciudadana','P_Cambio_Radical','P_Alianza_Verde','P_MIRA','P_de_la_U','Polo','P_Centro_Democratico','P_SOMOS','FARC','Colombia_Justa_Libres','SI_SE_PUEDE',"Union_con_Fortaleza","Todos_somos_Colombia",'Coalicion_Decencia','Voto_Blanco_Nacional','AICO','ASI','MAIS','SOBERANIA','MIA','RenovacionEtnica','CASIYOUREN','Voto_Blanco_Indigena','Nulos','No_Marcados',"Potencial votantes", "Referencia")
for i in localidades:
    temp=pd.read_csv("Votaciones_localidades/"+str(i))
    temp.index = Index
    df1[str(i[:-7])] = temp.iloc[:-1,1].apply(pd.to_numeric)

## Base de datos completa

In [23]:
#Suma el total de cada Item en Bogotá y genera un archivo CSV con la información
df1["Total Bogota"]=df1.sum(axis=1)
df1.to_csv("../Bogota/Votacion_Bogota_Unificada.csv")

## Circunscripcion nacional

In [24]:
dfn=df1[:-11]
dfn.to_csv("../Bogota/Circunscripcion_Nacional_Bogota.csv")

## Circunscripcion Indigena

In [25]:
dfi=df1[-11:-3]
dfi.to_csv("../Bogota/Circunscripcion_Indigena_Bogota.csv")